## Working with S&P500 data

In [2]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-03-27'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  505 of 505 completed

2 Failed downloads:
['GEV']: Exception("%ticker%: Data doesn't exist for startDate = 1459224000, endDate = 1711512000")
['SOLV']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2016-03-29 00:00:00 -> 2024-03-27)')


Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-03-29 A        38.160629   40.160000   40.220001   39.110001   39.349998   
           AAL      39.623764   41.349998   41.430000   40.770000   40.900002   
           AAPL     24.996071   26.920000   26.947500   26.219999   26.222500   
           ABBV     43.020931   57.049999   57.099998   55.220001   56.099998   
           ABT      37.127357   41.470001   41.529999   40.709999   40.750000   
...                       ...         ...         ...         ...         ...   
2024-03-26 XYL     127.900002  127.900002  128.490005  127.260002  127.559998   
           YUM     134.800003  134.800003  135.389999  134.589996  135.089996   
           ZBH     127.870003  127.870003  128.610001  127.849998  128.369995   
           ZBRA    291.380005  291.380005  301.519989  291.010010  300.000000   
           ZTS     165.669998  165.669998  169.229996  164.110001  169.210007   

Price                   volume  
date       ticker               
2016-03-29 A         1091200.0  
           AAL       6794000.0  
           AAPL    124760400.0  
           ABBV      4567900.0  
           ABT       4127000.0  
...                        ...  
2024-03-26 XYL        853400.0  
           YUM       1470400.0  
           ZBH       1080400.0  
           ZBRA       290600.0  
           ZTS       4550000.0  

[996171 rows x 6 columns]

# Indicators used
- Garman-Klass volatility
- Relative Strength Index
- Bollinger Bands
- Average True Range
- Moving Average Convergence Divergence
- Dollar Volume

In [3]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2016-03-29 A        38.160629   40.160000   40.220001   39.110001   39.349998   
           AAL      39.623764   41.349998   41.430000   40.770000   40.900002   
           AAPL     24.996071   26.920000   26.947500   26.219999   26.222500   
           ABBV     43.020931   57.049999   57.099998   55.220001   56.099998   
           ABT      37.127357   41.470001   41.529999   40.709999   40.750000   
...                       ...         ...         ...         ...         ...   
2024-03-26 XYL     127.900002  127.900002  128.490005  127.260002  127.559998   
           YUM     134.800003  134.800003  135.389999  134.589996  135.089996   
           ZBH     127.870003  127.870003  128.610001  127.849998  128.369995   
           ZBRA    291.380005  291.380005  301.519989  291.010010  300.000000   
           ZTS     165.669998  165.669998  169.229996  164.110001  169.210007   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2016-03-29 A         1091200.0          0.000028        NaN       NaN   
           AAL       6794000.0         -0.000259        NaN       NaN   
           AAPL    124760400.0         -0.000512        NaN       NaN   
           ABBV      4567900.0         -0.026659        NaN       NaN   
           ABT       4127000.0         -0.003150        NaN       NaN   
...                        ...               ...        ...       ...   
2024-03-26 XYL        853400.0          0.000044  61.098132  4.840505   
           YUM       1470400.0          0.000016  48.101000  4.908604   
           ZBH       1080400.0          0.000012  56.842472  4.826058   
           ZBRA       290600.0          0.000301  58.249245  5.621736   
           ZTS       4550000.0          0.000299  31.089087  5.085481   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2016-03-29 A            NaN       NaN       NaN       NaN      41.640879  
           AAL          NaN       NaN       NaN       NaN     269.203853  
           AAPL         NaN       NaN       NaN       NaN    3118.519799  
           ABBV         NaN       NaN       NaN       NaN     196.515312  
           ABT          NaN       NaN       NaN       NaN     153.224604  
...                     ...       ...       ...       ...            ...  
2024-03-26 XYL     4.856852  4.873199 -0.203452  0.831480     109.149861  
           YUM     4.933193  4.957782 -0.092908 -0.056967     198.209924  
           ZBH     4.846361  4.866665 -0.862913  0.238493     138.150751  
           ZBRA    5.660539  5.699342  0.065693  0.728707      84.675029  
           ZTS     5.192144  5.298807  1.001437 -2.901000     753.798492  

[996171 rows x 14 columns]

## Filter top 150 most liquid stocks in S&P500

In [4]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

data


dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2016-05-31 A          106.181063   43.731556         -0.000775  75.518272   
           AAL        304.649004   30.666256         -0.000261  32.311041   
           AAPL      3796.313254   23.321924         -0.001530  52.245045   
           ABBV       459.600256   47.910984         -0.027898  61.181409   
           ABT        414.171425   35.696785         -0.003539  50.100749   
...                          ...         ...               ...        ...   
2024-03-31 ABNB       742.723001  167.389999          0.000054  61.280283   
           CEG        501.200956  185.410004          0.002585  73.236660   
           GEHC       353.087329   89.070000          0.000155  54.999564   
           KVUE       383.999593   20.930000          0.000127  61.510678   
           VLTO       142.143533   89.260002          0.000063  61.561379   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2016-05-31 A       3.667166  3.746127  3.825087 -1.298754  0.466012  
           AAL     3.423399  3.470762  3.518124 -0.113268 -1.610672  
           AAPL    3.082426  3.142967  3.203509 -1.140877 -0.348241  
           ABBV    3.828151  3.870133  3.912116 -1.094209 -0.059967  
           ABT     3.535536  3.565837  3.596139 -1.208266 -0.653253  
...                     ...       ...       ...       ...       ...  
2024-03-31 ABNB    5.051744  5.100076  5.148408 -0.812586  0.860726  
           CEG     5.090804  5.160866  5.230927  4.957116  3.276785  
           GEHC    4.485599  4.523128  4.560657  0.104746  0.228792  
           KVUE    2.980758  3.042006  3.103254 -1.187162  1.523222  
           VLTO    4.468634  4.491973  4.515311 -1.208543  0.366962  

[47013 rows x 9 columns]

- Calculating 5 year rolling average dollar volume for each stock before filtering

In [5]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2017-04-30 AAL      41.276730         -0.000999  45.213626  3.691857   
           AAPL     34.050831         -0.001198  61.191720  3.535808   
           ABBV     52.139572         -0.021543  63.760509  3.931308   
           ABT      40.304489         -0.002468  48.977606  3.705412   
           ACN     113.002335         -0.001409  58.138673  4.674274   
...                       ...               ...        ...       ...   
2024-03-31 WMT      60.509998          0.000049  61.505206  4.091819   
           XOM     113.790001          0.000072  68.759599  4.642251   
           MRNA    107.410004          0.000372  56.969899  4.534684   
           UBER     77.910004          0.000240  56.107486  4.340786   
           ABNB    167.389999          0.000054  61.280283  5.051744   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2017-04-30 AAL     3.766755  3.841652  0.800603  0.660685  
           AAPL    3.551856  3.567903 -1.227554 -0.155656  
           ABBV    3.951106  3.970904 -1.429731 -0.101603  
           ABT     3.720749  3.736087 -1.339109 -0.289621  
           ACN     4.704352  4.734431 -1.139117 -0.193233  
...                     ...       ...       ...       ...  
2024-03-31 WMT     4.117984  4.144149  0.210401  1.484134  
           XOM     4.704803  4.767355 -0.030820  1.993959  
           MRNA    4.639621  4.744557 -0.457442  0.266602  
           UBER    4.377088  4.413390  1.194593  0.760472  
           ABNB    5.100076  5.148408 -0.812586  0.860726  

[12516 rows x 8 columns]